Install Dependences

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [3]:
# Set the path to your dataset folder
dataset_path = 'SKC'

In [ ]:
# Define the categories (subfolders in the dataset folder)
categories = ['besal cell carcinoma', 'melanoma', 'squamous cell carcinoma']

In [ ]:
# Define the image dimensions
IMG_SIZE = 224

In [ ]:
# Load the dataset
data = []
labels = []